# Sparseness-Optimized Feature Importance (tabular)

The algorithm finds a ranking of features $\mathbf{\Pi} = (\pi_1 \preceq \pi_2 \preceq, \dots , \preceq \pi_n)$ such that $\pi_i \in \mathbf{F}$ is the feature ranked in the $i$-th ranking position and $\mathbf{F}$ is the set of all problem features. In the feature importance problem, it holds that $\pi_i \preceq \pi_{i+1} \iff g(\pi_i) \geq g(\pi_{i+1})$ such that $g(\pi_i)$ is a function that returns model's performance after maginalizing up to the $i$-th feature in the ranking.

The proposed algorithm seeks to find $\mathbf{\Pi}$ using Genetic Algorithms in such a way that $g(\pi_1) \geq g(\pi_2) \geq \dots \geq g(\pi_n)$. It aims to minimize the area under the curve given by the points $(1, g(\pi_1)), (2, g(\pi_2)), \ldots, (n, g(\pi_n))$ so that the curve is monotonically decreasing. The curve should not have peaks, although plateaus may occur. This behavior is enforces with a regularized penalization term.

In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, classification_report
from sklearn.inspection import permutation_importance

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.6, style='whitegrid')

import pygad
import shap
import time
import os

import random
random.seed(42)

import warnings
warnings.simplefilter("ignore")

### Build and evaluate black-box

In [ ]:
def train_and_evaluate_model(dataset_path, test_size=0.2, random_state=42):
    # Load the dataset
    data = pd.read_csv(dataset_path)

    # Split the data into features (X) and the target variable (y)
    X = data.drop(data.columns[-1], axis=1)
    y = data[data.columns[-1]]

    # Normalize features
    scaler = MinMaxScaler()
    X_normalized = scaler.fit_transform(X)
    X = pd.DataFrame(X_normalized, columns=X.columns)

    # Encode target labels
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Create a GradientBoostingClassifier model
    model = GradientBoostingClassifier(random_state=random_state)

    # Define hyperparameters for grid search
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.1, 0.01, 0.001]
    }

    # Perform grid search cross-validation
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='f1_macro')
    grid_search.fit(X_train, y_train)

    # Get the best model
    best_model = grid_search.best_estimator_

    # Train the best model on the training data
    best_model.fit(X_train, y_train)

    # Predict on the test data
    y_pred = best_model.predict(X_test)

    # Evaluate the model on test data
    f1_macro = f1_score(y_test, y_pred, average='macro')

    return X_train, X_test, y_train, y_test, best_model, f1_macro

## SOFI

In [ ]:
def feature_flip(model, X, y, permutation):
    
    scores = []
    X_copy = X.copy()
    
    for feature in permutation:
        
        X_copy[X_copy.columns[feature]] = X_copy[X_copy.columns[feature]].mean()
        y_pred = model.predict(X_copy)
        f1 = f1_score(y, y_pred, average='macro')
        scores.append(f1)
        
    return scores

def fit_value(model, X, y, permutation):
    
    lambda_penalty = 0.1
    weights = np.arange(1.0, 0.0, -1/(len(permutation)+1))
    g_values = feature_flip(model, X, y, permutation)
    g_values = [baseline] + g_values
    
    area_under_curve = np.sum(weights * g_values)
    fitness = (area_under_curve + lambda_penalty * np.sum(np.maximum(np.diff(g_values), 0)))
    
    return fitness

def fitness_func(ga_instance, solution, solution_idx):
    
    auc = fit_value(best_model, X_test, y_test, solution)
    fitness = 1.0 / (auc + 0.000001) # for maximization
    return fitness

In [ ]:
def SOFI_permutation(model, X, y):
    ga_instance = pygad.GA(num_generations=20,
                           num_parents_mating=20,
                           fitness_func=fitness_func,
                           sol_per_pop=100,
                           num_genes=len(X.columns),
                           init_range_low=0,
                           init_range_high=len(X.columns)-1,
                           parent_selection_type='sss',
                           keep_parents=1,
                           crossover_type='single_point',
                           mutation_type='random',
                           mutation_probability=0.01,
                           gene_type=int,
                           gene_space=range(0, len(X.columns)),
                           allow_duplicate_genes=False,
                           parallel_processing=8)

    start_time = time.time()
    ga_instance.run()
    sofi_time = round(time.time() - start_time, 2)

    # Returning the details of the best solution.
    sofi_permutation, _, _ = ga_instance.best_solution(ga_instance.last_generation_fitness)

    return sofi_permutation, sofi_time

## SHAP

In [ ]:
def SHAP_permutation(model, X_train, X_test):
    start_time = time.time()
    explainer = shap.KernelExplainer(model.predict_proba, shap.kmeans(X_train.values, 20))
    shap_values = explainer.shap_values(X_test.values, silent=True)
    shap_time = round(time.time() - start_time, 2)

    # Calculate the average absolute SHAP values for each feature
    avg_abs_shap_values = np.mean(np.abs(shap_values[0]), axis=0)

    # Create a list of feature names
    feature_names = range(X_train.shape[1])  # Replace with the actual feature names if available

    # Create a dictionary to map feature names to their corresponding average SHAP values
    feature_importance = dict(zip(feature_names, avg_abs_shap_values))

    # Sort the features by their average SHAP values in descending order
    sorted_feature_importance = {k: v for k, v in sorted(
        feature_importance.items(), key=lambda item: item[1], reverse=True)}

    shap_permutation = list(sorted_feature_importance.keys())

    return shap_permutation, shap_time

## PFI

In [ ]:
def PFI_permutation(model, X, y, n_repeats=50):
    start_time = time.time()
    output = permutation_importance(model, X, y, n_repeats=n_repeats, random_state=42)
    pfi_time = round(time.time() - start_time, 2)

    perm_importance = output.importances_mean

    # Create a list of feature names
    feature_names = range(X.shape[1])  # Replace with the actual feature names if available

    # Create a dictionary to map feature names to their corresponding average permutation importances
    pfi_feature_importance = dict(zip(feature_names, perm_importance))

    # Sort the features by their average permutation importances in descending order
    sorted_feature_importance = {k: v for k, v in sorted(
        pfi_feature_importance.items(), key=lambda item: item[1], reverse=True)}

    pfi_permutation = list(sorted_feature_importance.keys())

    return pfi_permutation, pfi_time

### Plotting curves

In [ ]:
def plot_feature_flips(model, X, y, sofi_permutation, shap_permutation, pfi_permutation, baseline, dataset):
    fig, ax = plt.subplots()
    
    random_permutation = list(sofi_permutation) # making a copy
    random.shuffle(random_permutation)
    g_values_Rand = feature_flip(model, X, y, random_permutation)
    g_values_Rand = [baseline] + g_values_Rand

    g_values_SOFI = feature_flip(model, X, y, sofi_permutation)
    g_values_SOFI = [baseline] + g_values_SOFI

    g_values_SHAP = feature_flip(model, X, y, shap_permutation)
    g_values_SHAP = [baseline] + g_values_SHAP

    g_values_PFI = feature_flip(model, X, y, pfi_permutation)
    g_values_PFI = [baseline] + g_values_PFI

    x_data = np.array(range(0, len(shap_permutation) + 1))
    ax.plot(x_data, g_values_Rand, marker='', 
            markersize=10, linestyle='--', color='0.2', markerfacecolor='C3', label='Random')
    ax.plot(x_data, g_values_SHAP, marker='o', 
            markersize=10, linestyle='-', color='0.2', markerfacecolor='C1', label='SHAP')
    ax.plot(x_data, g_values_PFI, marker='o',
            markersize=10, linestyle='-', color='0.2', markerfacecolor='C2', label='PFI')
    ax.plot(x_data, g_values_SOFI, marker='o',
            markersize=10, linestyle='-', color='0.2', markerfacecolor='C0', label='SOFI')

    plt.gca().fill_between(x_data, g_values_SOFI, alpha=0.2, color='grey')
    min_data = np.min([g_values_SOFI, g_values_SHAP, g_values_PFI, g_values_Rand], axis=0)
    max_data = np.max([g_values_SOFI, g_values_SHAP, g_values_PFI, g_values_Rand], axis=0)
    ax.set_ylim(min(min_data)-0.02, max(max_data)+0.03)

    ax.set_xlabel('$\pi_i$')
    ax.set_ylabel('$g(\pi_i)$')
    ax.grid(False)
    ax.legend()

    plt.savefig(dataset.replace('.csv', '.pdf'), format='pdf', bbox_inches='tight')
#     plt.show()

In [ ]:
def calculate_auc_values(model, X, y, baseline, sofi_permutation, shap_permutation, pfi_permutation):
    
    weights = np.arange(1.0, 0.0, -1/(len(sofi_permutation)+1)) 
    
    g_values_SOFI = feature_flip(model, X, y, sofi_permutation)
    g_values_SOFI = [baseline] + g_values_SOFI
    area_under_curve = np.sum(weights * g_values_SOFI)
    auc_SOFI = round(area_under_curve, 2)

    g_values_SHAP = feature_flip(model, X, y, shap_permutation)
    g_values_SHAP = [baseline] + g_values_SHAP 
    area_under_curve = np.sum(weights * g_values_SHAP)
    auc_SHAP = round(area_under_curve, 2)

    g_values_PFI = feature_flip(model, X, y, pfi_permutation)
    g_values_PFI = [baseline] + g_values_PFI
    area_under_curve = np.sum(weights * g_values_PFI)
    auc_PFI = round(area_under_curve, 2)

    return auc_SOFI, auc_SHAP, auc_PFI

### Experiment for several tabular datasets

In [ ]:
# Function to list CSV files in a directory
def list_csv_files(directory):
    return [file for file in os.listdir(directory) if file.endswith('.csv')]

# Directory containing datasets
datasets_folder = 'data'

print("Dataset,SOFI_AUC,SHAP_AUC,PFI_AUC,SOFI_time,SHAP_time,PFI_time")

# Iterate over each dataset in the folder
for dataset_file in list_csv_files(datasets_folder):
    dataset_path = os.path.join(datasets_folder, dataset_file)

    # Train and evaluate the model for the current dataset
    X_train, X_test, y_train, y_test, best_model, baseline = train_and_evaluate_model(dataset_path)
    
    # Compute permutations for all feature importance methods
    sofi_permutation, sofi_time = SOFI_permutation(best_model, X_test, y_test)
    shap_permutation, shap_time = SHAP_permutation(best_model, X_train, X_test)
    pfi_permutation, pfi_time = PFI_permutation(best_model, X_test, y_test)
    
    # Plot feature flipping curves
    plot_feature_flips(best_model, X_test, y_test, sofi_permutation, shap_permutation, pfi_permutation, baseline, dataset_file)
    
    # Compute AUC for all permutations
    auc_SOFI, auc_SHAP, auc_PFI = calculate_auc_values(best_model, X_test, y_test, baseline, sofi_permutation, shap_permutation, pfi_permutation)
    
    print(dataset_file.replace(".csv","") + ","
         + str(auc_SOFI) + ","
         + str(auc_SHAP) + ","
         + str(auc_PFI) + ","
         + str(sofi_time) + ","
         + str(shap_time) + ","
         + str(pfi_time))

In [ ]:
# finished signal 
import winsound
winsound.Beep(1000, 1000)